# NLP Final Presentation - Airline Tweets LDA Model Demo

## Negative Topic Categorization
- Topic 1 --> Delay and Customer Service
- Topic 2 --> Baggage Issue
- Topic 3 --> Reschedule and Refund
- Topic 4 --> Phone and Online Booking
- Topic 5 --> Reservation Issue
- Topic 6 --> Seating Preferences
- Topic 7 --> Extra Charges
- Topic 8 --> Customer Experience

In [62]:
def predict_topic(tweet):
    from ipywidgets import interact
    import matplotlib.pyplot as plt
    import pandas as pd
    import seaborn as sns
    import numpy as np
    import sklearn
    from sklearn import preprocessing
    from sklearn.feature_extraction.text import CountVectorizer
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.pipeline import FeatureUnion
    from sklearn.feature_extraction import DictVectorizer
    from os import path
    from wordcloud import WordCloud, STOPWORDS 
    from PIL import Image
    from wordcloud import ImageColorGenerator
    import re

    import pickle
    import joblib
    import spacy

    import re
    import nltk
    from nltk.corpus import stopwords
    #nltk.download('wordnet')
    from nltk.stem import WordNetLemmatizer
    import string
    import numpy as np
    #nltk.download('punkt')
    from nltk import word_tokenize,sent_tokenize
    from nltk.stem.porter import PorterStemmer
    from textblob import TextBlob
    from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
    import datetime
    import warnings
    import gensim
    from gensim.utils import simple_preprocess
    from gensim.parsing.preprocessing import STOPWORDS
    from bs4 import BeautifulSoup
    from gensim.models import CoherenceModel
    from gensim import corpora, models

    lda_model4=models.LdaModel.load('/Users/jenniferwu/Desktop/MSBA/Spring 2020/NLP/final project/lda_model4.model')

    #preprocess data
    def preprocess(text):
        stopwords = set(STOPWORDS)
        stopwords.update(["american", "air","airline","thank","united","us","airways","virgin","america","jetblue","youre","extremely",
                     "usairway","usairways","flight","americanair","southwestair","southwestairlines","arbitrarily","dream","crazy",
                     "southwestairway","southwestairways","virginamerica","really","will","going","thanks","thankyou","passengersdont",
                     "please","got","let","take","help","already","never","now","told","guy","new","sure","still","amp","continue",
                     "plane","tell","ye","trying","yes","guy","much","appreciate", "thx","back","ok","good","credit","aacom",
                     "flying","love","great","awesome","see","nice","alway","httptcojwl26g6lrw","dontflythem","motherinlaw","night",
                     "nogearnotraining","seriously","didnt","coudnt","cant","wont","dont","wat","buffaloniagara","hasshe","morning",
                     "woulda","people","try","youve","youd","yours","flightled","tomorrow","today","wat","jfkyou","flite","cause",
                     "flightr","flight","need","hours","nooooo","like","doesnt","right","talk","tweet","mention","pbijfk","ridiculuous",
                     "wasnt","suppose","want","understand","come","work","worse","treat","think","know","worst","paulo","staduim",
                     "wouldnt","stay","away","wont","werent","happen","sorry","havent","tonight","drive","life","thing","aa951",
                     "whats","theyre","better","thats","allow","hope","stop","cool","niece","happy","word","customercant",
                     "suck","sunday","monday","tuesday","wednesday","thursday","friday","saturday","weekend","ruin","shouldnt",
                     "miami","los angeles","new york","chicago","dallas","apparently","itover","someones","savannah","lucymay",
                     "betterother","instead","look","hopefully","yesterday","antonio","unacceptable","folks","record",'arent',
                     "miss","hang","wrong","stick","grind","tarmac","theres","forget","terrible","clothe","terrible","break",
                     "actually","frustrate","correct","ridiculous","expect","different","pathetic","bother","follow","fault",
                     "impossible","point","cover","person","ask","speak","things","earlier","mean","select","minutes",
                     "unite","horrible","country","leave","speak","apologize","faster","hop","confuse","lose","flightd","hear",
                     "literally","years","surprise","bump","fail","compensate","hand","helpful","upset","friend","excuse","claim",
                     "situation","multiple","weather","choose","company","believe","question","kick","anymore","awful","delta",
                      "dozen","medical","completely","finally", "waste","shock","annoy","maybe","strand","mess","finally",
                      "plan","place","apology","center","plan","twitter","promise","prefer","count","maybe","shock","longer","meet",
                         "important","drop"])
        #stopwords.update([i for i in ts])
        # stopwords.update([str(i).lower() for i in cities.City]) #removing City names in US
        r = re.compile(r'(?<=\@)(\w+)') #remove words after tags --> usually twitter account
        ra = re.compile(r'(?<=\#)(\w+)') #remove words after hashtags
        ro = re.compile(r'(flt\d*)') #remove words after flight number
        names = r.findall(text.lower())
        hashtag = ra.findall(text.lower())
        flight = ro.findall(text.lower())
        lmtzr = WordNetLemmatizer()
        def stem_tokens(tokens, lemmatize):
            lemmatized = []
            for item in tokens:
                lemmatized.append(lmtzr.lemmatize(item,'v'))
            return lemmatized
        def deEmojify(inputString):
            return inputString.encode('ascii', 'ignore').decode('ascii')

        #doc = nlp(text)
        text = deEmojify(text)
        soup = BeautifulSoup(text)
        text = soup.get_text()
        text = "".join([ch.lower() for ch in text if ch not in string.punctuation])
        tokens = nltk.word_tokenize(text)
        tokens = [ch for ch in tokens if len(ch)>4] #remove words with character length below 2
        tokens = [ch for ch in tokens if len(ch)<=15] #remove words with character length above 15 
        lemm = stem_tokens(tokens, lmtzr)
        lemstop = [i for i in lemm if i not in stopwords]
        lemstopcl = [i for i in lemstop if i not in names]
        lemstopcl = [i for i in lemstopcl if i not in hashtag]
        lemstopcl = [i for i in lemstopcl if i not in flight]
        lemstopcl = [i for i in lemstopcl if not i.isdigit()]
        #lemstopcl1 = [i for i in lemstopcl if i not in t]
        return lemstopcl

    
    
    id2word = corpora.Dictionary.load('id2word.dict')

    bow_vector = id2word.doc2bow(preprocess(tweet))
    result = lda_model4.get_document_topics(bow_vector)
    resultdict = dict(result)
    orddict = sorted(resultdict, key=resultdict.get, reverse=True)
    
    Keymax = 1
    if resultdict[orddict[0]]-resultdict[orddict[1]] <=.08:
        Keymax +=orddict[1]
        print("Predicted topic: ",orddict[1]+1)
        print("Probability Score: ",resultdict[orddict[0]])
    else:
        Keymax +=orddict[0]
        print("Predicted topic: ",orddict[0]+1)
        print("Probability Score: ",resultdict[orddict[0]])


    if Keymax == 1:
        print('Delay and Customer Service')
    elif Keymax == 2:
        print('Baggage Issue')
    elif Keymax == 3:
        print('Reschedule and Refund')
    elif Keymax == 4:
        print('Phone and Online Booking')
    elif Keymax == 5:
        print('Reservation Issue')
    elif Keymax == 6:
        print('Seating Preferences')
    elif Keymax == 7:
        print('Extra Charges')
    else:
        print('Customer Experience')
    print("\n")

    
    df = pd.DataFrame(resultdict.items())
    df[0]=df[0]+1
    
    if Keymax==orddict[1]+1: 
        temp=df.iloc[orddict[0],1]
        df.iloc[orddict[0],[1]]=df.iloc[Keymax-1,1]
        df.iloc[Keymax-1,[1]]=temp
    else:
        df[1]=df[1]
    
    %matplotlib inline
    colors =  ['#3f5994','#8cbcfb','#c9e5fa','#f4dad0',"#f8bc99",'#f3aa84', '#f49c6c',"#c48771"]
    plt.style.use('ggplot')
    plt.figure(figsize=(20,12))
    plt.title("Topic Probability Score",size=48)
    plt.xticks(rotation= 20, size=30)
    plt.yticks(size=30)
    ax = sns.barplot(x=df[0],y=df[1],palette=colors)
    plt.ylabel('Probability Score', fontsize=35)
    plt.xlabel('Topics', fontsize=35)
    plt.show()
    
    for index, score in resultdict.items():
        print("Score: {}\n Topic: {}\n Keywords:{} \n".format(score, index+1, lda_model4.print_topic(index)))

In [63]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
interact_manual(predict_topic, tweet='abcd')

interactive(children=(Text(value='abcd', description='tweet'), Button(description='Run Interact', style=Button…

<function __main__.predict_topic(tweet)>